### Decision Trees

Decision Trees are versatile machine learning models that can perform both classification and regression tasks, including multioutput tasks. 

In [1]:
from sklearn.datasets import load_iris 
from sklearn.tree import DecisionTreeClassifier 

iris = load_iris()
X = iris.data[:, 2:] # petal length and width 
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth = 2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [3]:
# Visualize trained Decision Tree with graphviz 
from sklearn.tree import export_graphviz 

export_graphviz(
tree_clf, 
out_file = "./iris_tree.dot",
feature_names = iris.feature_names[2:],
class_names = iris.target_names,
rounded = True, 
filled = True)

**!** Unlike LogReg and SVMs, Decision Trees don't require feature scaling or centering at all

**Node attributes**: 
 * samples = how many training instances it applies to 
 * value = how many training instances of each class in this node applies to 
 * gini = impurity of the node. A node is considered pure (gini = 0) if all training instances it applies to belong to the same class. 
 

**Gini impurity**


$G_{i}$ = 1 - $\sum_{k = 1}^{n}$ $p_{i,k}^{2}$

$p_{i,k}$ = ratio of class k instances among the training instances in the i-th node.

**!** sklearn uses the CART algorithm which produces only binary trees (ie. nonleaf nodes always have two children). Other algorithms such as ID3 can produced Decision Trees with nodes that have more than two children.


**Estimating class probabilities** 

Decision Trees can also estimate the probability that an instance belongs to a particular class k. It first traverses the tree to find the leaf node for this instance, and then it returns the ratio of training instances of class k in this node. 

In [4]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [5]:
tree_clf.predict([[5, 1.5]])

array([1])

#### CART Training Algorithm for Classification

1. Split the training set into two subsets using a single feature k and a threshold $t{k}$ --> t and k are chosen so as to produce the purest subsets (weighted by their size). 

**Cost function for classification** 

$J(k, t_{k})$ = $\frac{m_{left}}{m}$ $G_{left}$ + $\frac{m_{right}}{m}$ $G_{right}$

 * $G_{left/right}$ = measures the impurity of the left/right subset 
 * $m_{left/right}$ = number of instances in the left/right subset
 
2. Split the two subsets using the same logic, recursively.

3. Stop recursion when it reaches the maximum depth (defined by the max_depth hyperparameter) or if it cannot find a split that will reduce impurity. 

CART is a greedy algorithm because it greedily searches for an optimum split at the top level, then repeats the process at each subsequent level. 

#### Gini impurity or Entropy

Gini impurity is the default, but can be substituted by entropy (criterion = 'entropy'). 

$H_{i}$ = - $\sum_{k = 1}^{n}$ $p_{i,k}log_{2}(p_{i,k})$

A set's entropy is 0 if it contains instances of only one class. 

 * Gini impurity is faster to compute but tends to isolate the most frequent class in its own branch of the tree. 
 
 * Entropy tends to produce slightly more balanced trees. 

#### Regularization hyperparameters 

Decision Trees many almost no assumption about the data and therefore have the risk of overfitting because the tree structure will adapt itself to the training data. Such a model is called **nonparametric** because the number of parameters is not predetermined prior to training, so the model structure is free to stick closely to the data. 

In contrast, a **parametric** model (eg. linear model) has a predetermined number of parameters, so its degree of freedom is limited, reducing the risk of overfitting but increasing the risk of underfitting. 

To avoid overfitting the training data, we can regularize Decision Trees: 

 * Reducing **max_depth** regularizes the model 
 * Increasing **min_samples_split** (minimum number of samples a node must have before it can be split), **min_samples_leaf** (minimum number of samples a leaf node must have), **min_weight_fraction_leaf** regularizes the model
 * Reducing **max_leaf_nodes** (maximum number of leaf nodes) and **max_features** (maximum number of features that are evaluated for splitting each node) regularizes the model 
 

**Pruning** 

Alternatively, other algorithms work by first training a Decision Tree without restrictions and then pruning unnecessary nodes. --> $\chi^{2}$ test is used to estimate the probability that the improvement due to pruning is purely the result of chance (null hypothesis). If this probability (p-value) is higher than a given threshold, then the node is considered unnecessary and its children are deleted. 

#### Regression 

In [6]:
from sklearn.tree import DecisionTreeRegressor 

tree_reg = DecisionTreeRegressor(max_depth = 2)
tree_reg.fit(X, y)

DecisionTreeRegressor(max_depth=2)

The prediction is the average target value of the training instances associated with the leaf node of interest. 

**Cost function for regression**

$J(k, t_{k})$ = $\frac{m_{left}}{m}$ $MSE_{left}$ + $\frac{m_{right}}{m}$ $MSE_{right}$

Instead of trying to split the training set in a way that minimizes impurity, it now tries to split the training set in a way that minimizes the MSE. 



#### Instability 

Decision trees like orthogonal decision boundaries, which makes them sensitive to training set rotation. One way to limit this problem is to use **Principal Component Analysis**, which results in a better orientation of the training data. 

#### End of notebook